# Setup Database e Tabelas - MKL Bank

Este notebook cria o database, schema e todas as tabelas necessárias para o sistema bancário MKL baseado nos schemas dos arquivos CSV da pasta `sample_sintetic_data`.

## Objetivo
- Criar o database FSI (Financial Services Industry)
- Criar o schema de destino
- Analisar os arquivos CSV para definir estruturas de tabelas
- Criar todas as tabelas com os tipos de dados apropriados
- Verificar a criação das tabelas

In [16]:
# Import Required Libraries
import os
import pandas as pd
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import logging
from datetime import datetime
import numpy as np
from dotenv import load_dotenv 
import warnings
# Carregar variáveis do arquivo .env de um diretório específico
env_path = "../env_files/.env"
load_dotenv(dotenv_path=env_path)
warnings.filterwarnings('ignore')

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


In [17]:
# Database Configuration Setup

# Configuração melhorada - conecta primeiro ao postgres padrão
DB_CONFIG = {
    'host': os.getenv("PGHOST", "localhost"),
    'port': int(os.getenv("PGPORT", "5432")),  # Converter para int
    'user': os.getenv("PGUSER", "postgres"),
    'password': os.getenv("PGPASSWORD", "postgres"),
    'database': "postgres",  # SEMPRE conectar ao postgres padrão primeiro
    'sslmode': os.getenv("PGSSLMODE", "require"),
    'connect_timeout': 30,  # Timeout de 30 segundos
    'application_name': 'MKL-Bank-Setup'  # Identificação da aplicação
}

TARGETSCHEMA = os.getenv("TARGETSCHEMA", "public")
# O database que queremos criar/usar é o mkl_bank do .env
TARGET_DB = os.getenv("PGDATABASE", "mkl_bank")

# Caminho para os arquivos CSV
DATA_PATH = "../assets/sample_sintetic_data/"

print(f"🔧 Configuração do Database:")
print(f"   Host: {DB_CONFIG['host']}")
print(f"   Port: {DB_CONFIG['port']}")
print(f"   User: {DB_CONFIG['user']}")
print(f"   Initial Database: {DB_CONFIG['database']} (conexão admin)")
print(f"   Target Database: {TARGET_DB} (será criado se não existir)")
print(f"   Target Schema: {TARGETSCHEMA}")
print(f"   SSL Mode: {DB_CONFIG['sslmode']}")
print(f"   Connect Timeout: {DB_CONFIG['connect_timeout']}s")
print(f"   Data Path: {DATA_PATH}")

🔧 Configuração do Database:
   Host: pgdtgov5wm5wmynq.postgres.database.azure.com
   Port: 5432
   User: pgadmin
   Initial Database: postgres (conexão admin)
   Target Database: mkl_bank (será criado se não existir)
   Target Schema: core_bank
   SSL Mode: require
   Connect Timeout: 30s
   Data Path: ../assets/sample_sintetic_data/


In [18]:
# Teste de Conectividade Melhorada
print("\n🔗 TESTE DE CONECTIVIDADE FINAL")
print("=" * 50)

# Verificar se temos uma configuração funcional
if 'working_config' in globals() and working_config:
    print("✅ Configuração funcional detectada!")
    print(f"   Host: {working_config['host']}")
    print(f"   Port: {working_config['port']}")
    print(f"   Database: {working_config['database']}")
    print(f"   SSL Mode: {working_config.get('sslmode', 'N/A')}")
    print(f"   Timeout: {working_config.get('connect_timeout', 'N/A')}s")
    
    # Teste final de conexão
    try:
        conn = psycopg2.connect(**working_config)
        cursor = conn.cursor()
        
        # Teste básico
        cursor.execute("SELECT current_database(), current_user, version();")
        db_name, username, version = cursor.fetchone()
        
        print(f"\n📊 Informações da Conexão:")
        print(f"   Database Atual: {db_name}")
        print(f"   Usuário: {username}")
        print(f"   Versão: {version.split(',')[0]}")
        
        # Verificar se o database mkl_bank já existe
        cursor.execute("""
            SELECT 1 FROM pg_database 
            WHERE datname = %s;
        """, (TARGET_DB,))
        
        db_exists = cursor.fetchone()
        if db_exists:
            print(f"   ⚠️  Database '{TARGET_DB}' já existe!")
        else:
            print(f"   ℹ️  Database '{TARGET_DB}' precisa ser criado")
        
        cursor.close()
        conn.close()
        
        print("\n✅ Conectividade confirmada! Pronto para próximas etapas.")
        
    except Exception as e:
        print(f"❌ Erro na conexão final: {e}")
        
else:
    print("❌ Nenhuma configuração funcional encontrada.")
    print("Verifique os pré-requisitos de rede e autenticação.")


🔗 TESTE DE CONECTIVIDADE FINAL
✅ Configuração funcional detectada!
   Host: pgdtgov5wm5wmynq.postgres.database.azure.com
   Port: 5432
   Database: postgres
   SSL Mode: N/A
   Timeout: N/As

📊 Informações da Conexão:
   Database Atual: postgres
   Usuário: pgadmin
   Versão: PostgreSQL 16.9 on x86_64-pc-linux-gnu
   ⚠️  Database 'mkl_bank' já existe!

✅ Conectividade confirmada! Pronto para próximas etapas.

📊 Informações da Conexão:
   Database Atual: postgres
   Usuário: pgadmin
   Versão: PostgreSQL 16.9 on x86_64-pc-linux-gnu
   ⚠️  Database 'mkl_bank' já existe!

✅ Conectividade confirmada! Pronto para próximas etapas.


In [19]:
# Criação do Database mkl_bank e Conexão Final
def get_default_connection():
    """Conecta ao database postgres padrão para operações administrativas"""
    return psycopg2.connect(**working_config)

def get_target_connection():
    """Conecta ao database mkl_bank após criá-lo"""
    target_config = working_config.copy()
    target_config['database'] = TARGET_DB
    return psycopg2.connect(**target_config)

# Criar o database mkl_bank se não existir
print("\n🏗️ CRIANDO DATABASE mkl_bank")
print("=" * 40)

try:
    # Conectar ao postgres padrão
    default_conn = get_default_connection()
    default_conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    
    with default_conn.cursor() as cursor:
        # Verificar se já existe
        cursor.execute("SELECT 1 FROM pg_database WHERE datname = %s", (TARGET_DB,))
        if cursor.fetchone():
            print(f"ℹ️  Database '{TARGET_DB}' já existe")
        else:
            # Criar o database
            cursor.execute(f'CREATE DATABASE "{TARGET_DB}"')
            print(f"✅ Database '{TARGET_DB}' criado com sucesso")
    
    default_conn.close()
    
    # Agora conectar ao database mkl_bank
    print(f"\n🔌 CONECTANDO AO DATABASE {TARGET_DB}")
    target_conn = get_target_connection()
    
    with target_conn.cursor() as cursor:
        cursor.execute("SELECT current_database(), current_user;")
        current_db, current_user = cursor.fetchone()
        print(f"✅ Conectado a: {current_db} como {current_user}")
    
    # Manter a conexão disponível para próximas células
    conn = target_conn
    
    print(f"\n🎯 PRONTO! Variável 'conn' configurada para {TARGET_DB}")
    
except Exception as e:
    print(f"❌ Erro: {e}")
    # Manter conexão padrão se houver problema
    conn = get_default_connection()
    print("⚠️  Usando conexão padrão ao postgres")


🏗️ CRIANDO DATABASE mkl_bank
ℹ️  Database 'mkl_bank' já existe

🔌 CONECTANDO AO DATABASE mkl_bank
ℹ️  Database 'mkl_bank' já existe

🔌 CONECTANDO AO DATABASE mkl_bank
✅ Conectado a: mkl_bank como pgadmin

🎯 PRONTO! Variável 'conn' configurada para mkl_bank
✅ Conectado a: mkl_bank como pgadmin

🎯 PRONTO! Variável 'conn' configurada para mkl_bank


In [20]:
# Database Connection Functions
def get_default_connection():
    """Conecta ao database padrão para operações administrativas"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        return conn
    except Exception as e:
        logger.error(f"Erro ao conectar ao database padrão: {e}")
        raise

def get_target_connection():
    """Conecta ao database de destino"""
    try:
        target_config = DB_CONFIG.copy()
        target_config['database'] = TARGET_DB
        conn = psycopg2.connect(**target_config)
        return conn
    except Exception as e:
        logger.error(f"Erro ao conectar ao database de destino: {e}")
        raise

def execute_sql(connection, sql, fetch=False):
    """Executa SQL e retorna resultado se necessário"""
    try:
        cursor = connection.cursor()
        cursor.execute(sql)
        
        if fetch:
            result = cursor.fetchall()
            cursor.close()
            return result
        else:
            connection.commit()
            cursor.close()
            return True
            
    except Exception as e:
        logger.error(f"Erro ao executar SQL: {e}")
        raise

print("✅ Funções de conexão definidas!")



✅ Funções de conexão definidas!


In [21]:
# Create Target Database
def create_database():
    """Cria o database de destino se não existir"""
    try:
        logger.info(f"Verificando se database '{TARGET_DB}' existe...")
        
        # Conecta ao database padrão
        conn = get_default_connection()
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        
        # Verifica se o database já existe
        check_db_sql = "SELECT 1 FROM pg_catalog.pg_database WHERE datname = %s"
        cursor = conn.cursor()
        cursor.execute(check_db_sql, (TARGET_DB,))
        exists = cursor.fetchone()
        
        if not exists:
            # Cria o database
            create_db_sql = f'CREATE DATABASE "{TARGET_DB}"'
            cursor.execute(create_db_sql)
            logger.info(f"✅ Database '{TARGET_DB}' criado com sucesso!")
        else:
            logger.info(f"ℹ️ Database '{TARGET_DB}' já existe.")
            
        cursor.close()
        conn.close()
        
        return True
        
    except Exception as e:
        logger.error(f"❌ Erro ao criar database: {e}")
        raise

# Executar criação do database
create_database()

2025-09-24 22:58:55,081 - INFO - Verificando se database 'mkl_bank' existe...
2025-09-24 22:59:01,513 - INFO - ℹ️ Database 'mkl_bank' já existe.
2025-09-24 22:59:01,513 - INFO - ℹ️ Database 'mkl_bank' já existe.


True

In [22]:
# Create Schema
def create_schema():
    """Cria o schema de destino se não existir"""
    try:
        logger.info(f"Criando schema '{TARGETSCHEMA}' no database '{TARGET_DB}'...")
        
        conn = get_target_connection()
        
        # Cria o schema
        create_schema_sql = f'CREATE SCHEMA IF NOT EXISTS "{TARGETSCHEMA}"'
        execute_sql(conn, create_schema_sql)
        
        logger.info(f"✅ Schema '{TARGETSCHEMA}' criado/verificado com sucesso!")
        
        conn.close()
        return True
        
    except Exception as e:
        logger.error(f"❌ Erro ao criar schema: {e}")
        raise

# Executar criação do schema
create_schema()

2025-09-24 22:59:01,525 - INFO - Criando schema 'core_bank' no database 'mkl_bank'...
2025-09-24 22:59:07,657 - INFO - ✅ Schema 'core_bank' criado/verificado com sucesso!
2025-09-24 22:59:07,657 - INFO - ✅ Schema 'core_bank' criado/verificado com sucesso!


True

In [23]:
# Analyze CSV File Schemas
def analyze_csv_files():
    """Analisa os arquivos CSV para determinar os schemas das tabelas"""
    
    csv_files = [
        'agencias.csv',
        'clientes.csv', 
        'contas.csv',
        'cartoes.csv',
        'chaves_pix.csv',
        'transacoes.csv'
    ]
    
    schemas = {}
    
    for csv_file in csv_files:
        try:
            file_path = os.path.join(DATA_PATH, csv_file)
            
            if not os.path.exists(file_path):
                logger.warning(f"⚠️ Arquivo não encontrado: {file_path}")
                continue
                
            # Lê uma amostra do arquivo para análise
            df = pd.read_csv(file_path, nrows=100)
            table_name = csv_file.replace('.csv', '')
            
            logger.info(f"📊 Analisando {csv_file} ({len(df)} amostras)")
            print(f"\nTabela: {table_name}")
            print(f"Colunas: {list(df.columns)}")
            
            # Analisa tipos de dados
            column_types = {}
            for col in df.columns:
                col_data = df[col].dropna()
                
                if col_data.empty:
                    column_types[col] = "TEXT"
                    continue
                
                # Determina tipo baseado no conteúdo e nome da coluna
                if col in ['id_cliente', 'codigo_agencia', 'id_conta', 'cartao_id', 'agencia_id']:
                    column_types[col] = "BIGINT"
                elif col in ['chave_pix_id']:
                    column_types[col] = "SERIAL"
                elif 'id' in col.lower() and 'transacao' in col.lower():
                    column_types[col] = "VARCHAR(30)"
                elif 'data_' in col or 'created_at' in col:
                    column_types[col] = "TIMESTAMP"
                elif col in ['saldo', 'valor', 'limite_credito', 'limite']:
                    column_types[col] = "DECIMAL(15,2)"
                elif col in ['cpf']:
                    column_types[col] = "VARCHAR(14)"
                elif col in ['numero_cartao']:
                    column_types[col] = "VARCHAR(16)"
                elif col in ['codigo_barras', 'codigo_boleto']:
                    column_types[col] = "VARCHAR(100)"
                elif col in ['telefone']:
                    column_types[col] = "VARCHAR(20)"
                elif col in ['email']:
                    column_types[col] = "VARCHAR(200)"
                elif col in ['nome', 'nome_agencia', 'estabelecimento', 'beneficiario', 'depositante']:
                    column_types[col] = "VARCHAR(200)"
                elif col in ['endereco', 'descricao']:
                    column_types[col] = "TEXT"
                elif col in ['cidade']:
                    column_types[col] = "VARCHAR(100)"
                elif col in ['estado']:
                    column_types[col] = "CHAR(2)"
                elif col in ['cep']:
                    column_types[col] = "VARCHAR(10)"
                elif col in ['status', 'tipo', 'genero']:
                    column_types[col] = "CHAR(1)"
                elif col in ['tipo_cartao', 'tipo_chave', 'tipo_transferencia', 'tipo_deposito', 'categoria', 'operacao']:
                    column_types[col] = "VARCHAR(50)"
                elif col in ['chave_pix', 'chave_pix_destino', 'chave_pix_origem', 'valor_chave']:
                    column_types[col] = "VARCHAR(200)"
                else:
                    # Análise automática para outros campos
                    sample_val = str(col_data.iloc[0]) if len(col_data) > 0 else ""
                    if len(sample_val) > 100:
                        column_types[col] = "TEXT"
                    elif len(sample_val) > 50:
                        column_types[col] = "VARCHAR(200)"
                    else:
                        column_types[col] = "VARCHAR(100)"
            
            schemas[table_name] = {
                'columns': column_types,
                'sample_data': df.head(3)
            }
            
            print(f"Schema definido: {column_types}")
            
        except Exception as e:
            logger.error(f"❌ Erro ao analisar {csv_file}: {e}")
    
    return schemas

# Executar análise
table_schemas = analyze_csv_files()
print(f"\n✅ Análise concluída! {len(table_schemas)} tabelas analisadas.")

2025-09-24 22:59:07,677 - INFO - 📊 Analisando agencias.csv (100 amostras)
2025-09-24 22:59:07,682 - INFO - 📊 Analisando clientes.csv (100 amostras)
2025-09-24 22:59:07,682 - INFO - 📊 Analisando clientes.csv (100 amostras)
2025-09-24 22:59:07,688 - INFO - 📊 Analisando contas.csv (100 amostras)
2025-09-24 22:59:07,695 - INFO - 📊 Analisando cartoes.csv (100 amostras)
2025-09-24 22:59:07,688 - INFO - 📊 Analisando contas.csv (100 amostras)
2025-09-24 22:59:07,695 - INFO - 📊 Analisando cartoes.csv (100 amostras)
2025-09-24 22:59:07,700 - INFO - 📊 Analisando chaves_pix.csv (100 amostras)
2025-09-24 22:59:07,707 - INFO - 📊 Analisando transacoes.csv (100 amostras)
2025-09-24 22:59:07,700 - INFO - 📊 Analisando chaves_pix.csv (100 amostras)
2025-09-24 22:59:07,707 - INFO - 📊 Analisando transacoes.csv (100 amostras)



Tabela: agencias
Colunas: ['codigo_agencia', 'nome', 'endereco', 'cidade', 'estado', 'telefone']
Schema definido: {'codigo_agencia': 'BIGINT', 'nome': 'VARCHAR(200)', 'endereco': 'TEXT', 'cidade': 'VARCHAR(100)', 'estado': 'CHAR(2)', 'telefone': 'VARCHAR(20)'}

Tabela: clientes
Colunas: ['id_cliente', 'cpf', 'nome', 'data_nascimento', 'genero', 'email', 'telefone']
Schema definido: {'id_cliente': 'BIGINT', 'cpf': 'VARCHAR(14)', 'nome': 'VARCHAR(200)', 'data_nascimento': 'TIMESTAMP', 'genero': 'CHAR(1)', 'email': 'VARCHAR(200)', 'telefone': 'VARCHAR(20)'}

Tabela: contas
Colunas: ['id_conta', 'codigo_agencia', 'id_cliente', 'saldo', 'data_abertura', 'status']
Schema definido: {'id_conta': 'BIGINT', 'codigo_agencia': 'BIGINT', 'id_cliente': 'BIGINT', 'saldo': 'DECIMAL(15,2)', 'data_abertura': 'TIMESTAMP', 'status': 'CHAR(1)'}

Tabela: cartoes
Colunas: ['numero_cartao', 'id_cliente', 'data_emissao', 'limite_credito', 'status']
Schema definido: {'numero_cartao': 'VARCHAR(16)', 'id_cliente

In [24]:
# Generate Table Creation Scripts
def generate_create_table_scripts(schemas):
    """Gera scripts SQL para criação das tabelas"""
    
    create_scripts = {}
    
    # Ordem de criação (tabelas com FK devem vir depois)
    table_order = ['agencias', 'clientes', 'contas', 'cartoes', 'chaves_pix', 'transacoes']
    
    for table_name in table_order:
        if table_name not in schemas:
            continue
            
        columns = schemas[table_name]['columns']
        
        # Monta o script CREATE TABLE
        script_lines = [f'CREATE TABLE IF NOT EXISTS "{TARGETSCHEMA}".{table_name} (']
        
        column_definitions = []
        
        for col_name, col_type in columns.items():
            definition = f'    "{col_name}" {col_type}'
            
            # Adiciona constraints
            if col_name in ['id_cliente', 'codigo_agencia', 'id_conta', 'numero_cartao', 'id_transacao', 'chave_pix_id']:
                definition += ' PRIMARY KEY'
            elif col_name == 'cpf':
                definition += ' UNIQUE NOT NULL'
            elif 'id' in col_name.lower() and col_name != 'chave_pix_id':
                definition += ' NOT NULL'
            elif col_name in ['nome', 'chave_pix']:
                definition += ' NOT NULL'
                
            column_definitions.append(definition)
        
        # Adiciona foreign keys
        if table_name == 'contas':
            column_definitions.append(f'    FOREIGN KEY ("codigo_agencia") REFERENCES "{TARGETSCHEMA}".agencias("codigo_agencia")')
            column_definitions.append(f'    FOREIGN KEY ("id_cliente") REFERENCES "{TARGETSCHEMA}".clientes("id_cliente")')
        elif table_name == 'cartoes':
            column_definitions.append(f'    FOREIGN KEY ("id_cliente") REFERENCES "{TARGETSCHEMA}".clientes("id_cliente")')
        elif table_name == 'chaves_pix':
            column_definitions.append(f'    FOREIGN KEY ("id_cliente") REFERENCES "{TARGETSCHEMA}".clientes("id_cliente")')
        elif table_name == 'transacoes':
            column_definitions.append(f'    FOREIGN KEY ("id_conta") REFERENCES "{TARGETSCHEMA}".contas("id_conta")')
        
        script_lines.append(',\n'.join(column_definitions))
        script_lines.append(');')
        
        create_scripts[table_name] = '\n'.join(script_lines)
        
        logger.info(f"📝 Script gerado para tabela: {table_name}")
    
    return create_scripts

# Gerar scripts
table_scripts = generate_create_table_scripts(table_schemas)

# Exibir scripts gerados
print("📋 Scripts de criação de tabelas:")
print("=" * 60)
for table_name, script in table_scripts.items():
    print(f"\n-- Tabela: {table_name}")
    print(script)

2025-09-24 22:59:07,721 - INFO - 📝 Script gerado para tabela: agencias
2025-09-24 22:59:07,723 - INFO - 📝 Script gerado para tabela: clientes
2025-09-24 22:59:07,724 - INFO - 📝 Script gerado para tabela: contas
2025-09-24 22:59:07,725 - INFO - 📝 Script gerado para tabela: cartoes
2025-09-24 22:59:07,725 - INFO - 📝 Script gerado para tabela: chaves_pix
2025-09-24 22:59:07,726 - INFO - 📝 Script gerado para tabela: transacoes
2025-09-24 22:59:07,723 - INFO - 📝 Script gerado para tabela: clientes
2025-09-24 22:59:07,724 - INFO - 📝 Script gerado para tabela: contas
2025-09-24 22:59:07,725 - INFO - 📝 Script gerado para tabela: cartoes
2025-09-24 22:59:07,725 - INFO - 📝 Script gerado para tabela: chaves_pix
2025-09-24 22:59:07,726 - INFO - 📝 Script gerado para tabela: transacoes


📋 Scripts de criação de tabelas:

-- Tabela: agencias
CREATE TABLE IF NOT EXISTS "core_bank".agencias (
    "codigo_agencia" BIGINT PRIMARY KEY,
    "nome" VARCHAR(200) NOT NULL,
    "endereco" TEXT,
    "cidade" VARCHAR(100) NOT NULL,
    "estado" CHAR(2),
    "telefone" VARCHAR(20)
);

-- Tabela: clientes
CREATE TABLE IF NOT EXISTS "core_bank".clientes (
    "id_cliente" BIGINT PRIMARY KEY,
    "cpf" VARCHAR(14) UNIQUE NOT NULL,
    "nome" VARCHAR(200) NOT NULL,
    "data_nascimento" TIMESTAMP,
    "genero" CHAR(1),
    "email" VARCHAR(200),
    "telefone" VARCHAR(20)
);

-- Tabela: contas
CREATE TABLE IF NOT EXISTS "core_bank".contas (
    "id_conta" BIGINT PRIMARY KEY,
    "codigo_agencia" BIGINT PRIMARY KEY,
    "id_cliente" BIGINT PRIMARY KEY,
    "saldo" DECIMAL(15,2),
    "data_abertura" TIMESTAMP,
    "status" CHAR(1),
    FOREIGN KEY ("codigo_agencia") REFERENCES "core_bank".agencias("codigo_agencia"),
    FOREIGN KEY ("id_cliente") REFERENCES "core_bank".clientes("id_cliente

In [25]:
# Create Database Schema and Tables
def create_tables(scripts):
    """Executa os scripts de criação das tabelas"""
    
    try:
        conn = get_target_connection()
        
        logger.info("🚀 Iniciando criação das tabelas...")
        
        # Ordem de criação
        table_order = ['agencias', 'clientes', 'contas', 'cartoes', 'chaves_pix', 'transacoes']
        
        for table_name in table_order:
            if table_name in scripts:
                logger.info(f"📝 Criando tabela: {table_name}")
                
                try:
                    execute_sql(conn, scripts[table_name])
                    logger.info(f"✅ Tabela '{table_name}' criada com sucesso!")
                    
                except Exception as e:
                    logger.error(f"❌ Erro ao criar tabela '{table_name}': {e}")
                    raise
        
        conn.close()
        logger.info("🎉 Todas as tabelas foram criadas com sucesso!")
        
    except Exception as e:
        logger.error(f"❌ Erro na criação das tabelas: {e}")
        raise

# Executar criação das tabelas
create_tables(table_scripts)

2025-09-24 22:59:09,808 - INFO - 🚀 Iniciando criação das tabelas...
2025-09-24 22:59:09,809 - INFO - 📝 Criando tabela: agencias
2025-09-24 22:59:09,809 - INFO - 📝 Criando tabela: agencias
2025-09-24 22:59:11,069 - INFO - ✅ Tabela 'agencias' criada com sucesso!
2025-09-24 22:59:11,070 - INFO - 📝 Criando tabela: clientes
2025-09-24 22:59:11,069 - INFO - ✅ Tabela 'agencias' criada com sucesso!
2025-09-24 22:59:11,070 - INFO - 📝 Criando tabela: clientes
2025-09-24 22:59:11,552 - INFO - ✅ Tabela 'clientes' criada com sucesso!
2025-09-24 22:59:11,553 - INFO - 📝 Criando tabela: contas
2025-09-24 22:59:11,552 - INFO - ✅ Tabela 'clientes' criada com sucesso!
2025-09-24 22:59:11,553 - INFO - 📝 Criando tabela: contas
2025-09-24 22:59:12,078 - INFO - ✅ Tabela 'contas' criada com sucesso!
2025-09-24 22:59:12,079 - INFO - 📝 Criando tabela: cartoes
2025-09-24 22:59:12,078 - INFO - ✅ Tabela 'contas' criada com sucesso!
2025-09-24 22:59:12,079 - INFO - 📝 Criando tabela: cartoes
2025-09-24 22:59:12,533 

In [26]:
# 🔧 CORREÇÃO DOS DDLs E CRIAÇÃO DAS TABELAS
print("🔧 Corrigindo DDLs e criando tabelas...")

# DDLs corrigidos com chaves primárias únicas
corrected_ddls = {
    "agencias": """
    CREATE TABLE IF NOT EXISTS "core_bank".agencias (
        "codigo_agencia" BIGINT PRIMARY KEY,
        "nome" VARCHAR(200) NOT NULL,
        "endereco" TEXT,
        "cidade" VARCHAR(100) NOT NULL,
        "estado" CHAR(2),
        "telefone" VARCHAR(20)
    );
    """,
    
    "clientes": """
    CREATE TABLE IF NOT EXISTS "core_bank".clientes (
        "id_cliente" BIGINT PRIMARY KEY,
        "cpf" VARCHAR(14) UNIQUE NOT NULL,
        "nome" VARCHAR(200) NOT NULL,
        "data_nascimento" TIMESTAMP,
        "genero" CHAR(1),
        "email" VARCHAR(200),
        "telefone" VARCHAR(20)
    );
    """,
    
    "contas": """
    CREATE TABLE IF NOT EXISTS "core_bank".contas (
        "id_conta" BIGINT PRIMARY KEY,
        "codigo_agencia" BIGINT NOT NULL,
        "id_cliente" BIGINT NOT NULL,
        "saldo" DECIMAL(15,2) DEFAULT 0,
        "data_abertura" TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        "status" CHAR(1) DEFAULT 'A',
        FOREIGN KEY ("codigo_agencia") REFERENCES "core_bank".agencias("codigo_agencia"),
        FOREIGN KEY ("id_cliente") REFERENCES "core_bank".clientes("id_cliente")
    );
    """,
    
    "cartoes": """
    CREATE TABLE IF NOT EXISTS "core_bank".cartoes (
        "numero_cartao" VARCHAR(16) PRIMARY KEY,
        "id_cliente" BIGINT NOT NULL,
        "data_emissao" TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        "limite_credito" DECIMAL(15,2) DEFAULT 0,
        "status" CHAR(1) DEFAULT 'A',
        FOREIGN KEY ("id_cliente") REFERENCES "core_bank".clientes("id_cliente")
    );
    """,
    
    "chaves_pix": """
    CREATE TABLE IF NOT EXISTS "core_bank".chaves_pix (
        "id_chave_pix" SERIAL PRIMARY KEY,
        "id_cliente" BIGINT NOT NULL,
        "id_conta" BIGINT NOT NULL,
        "tipo_chave" VARCHAR(50) NOT NULL,
        "valor_chave" VARCHAR(200) UNIQUE NOT NULL,
        "data_cadastro" TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        "status" CHAR(1) DEFAULT 'A',
        FOREIGN KEY ("id_cliente") REFERENCES "core_bank".clientes("id_cliente"),
        FOREIGN KEY ("id_conta") REFERENCES "core_bank".contas("id_conta")
    );
    """,
    
    "transacoes": """
    CREATE TABLE IF NOT EXISTS "core_bank".transacoes (
        "id_transacao" VARCHAR(30) PRIMARY KEY,
        "id_conta" BIGINT NOT NULL,
        "data_transacao" TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        "valor" DECIMAL(15,2) NOT NULL,
        "tipo" CHAR(1) NOT NULL,
        "descricao" TEXT,
        FOREIGN KEY ("id_conta") REFERENCES "core_bank".contas("id_conta")
    );
    """
}

# Função para criar tabelas com ordem correta (considerando foreign keys)
def create_tables_ordered():
    table_order = ["agencias", "clientes", "contas", "cartoes", "chaves_pix", "transacoes"]
    
    for table_name in table_order:
        print(f"📝 Criando tabela: {table_name}")
        try:
            execute_sql(conn, corrected_ddls[table_name])
            print(f"✅ Tabela '{table_name}' criada com sucesso!")
        except Exception as e:
            print(f"❌ Erro ao criar tabela '{table_name}': {e}")
            # Se a tabela já existe, continuar
            if "already exists" in str(e):
                print(f"ℹ️  Tabela '{table_name}' já existe, continuando...")
                continue
            else:
                raise e

# Executar criação das tabelas
create_tables_ordered()
print("\n🎉 CRIAÇÃO DE TABELAS CONCLUÍDA!")

🔧 Corrigindo DDLs e criando tabelas...
📝 Criando tabela: agencias
✅ Tabela 'agencias' criada com sucesso!
📝 Criando tabela: clientes
✅ Tabela 'agencias' criada com sucesso!
📝 Criando tabela: clientes
✅ Tabela 'clientes' criada com sucesso!
📝 Criando tabela: contas
✅ Tabela 'clientes' criada com sucesso!
📝 Criando tabela: contas
✅ Tabela 'contas' criada com sucesso!
📝 Criando tabela: cartoes
✅ Tabela 'contas' criada com sucesso!
📝 Criando tabela: cartoes
✅ Tabela 'cartoes' criada com sucesso!
📝 Criando tabela: chaves_pix
✅ Tabela 'cartoes' criada com sucesso!
📝 Criando tabela: chaves_pix
✅ Tabela 'chaves_pix' criada com sucesso!
📝 Criando tabela: transacoes
✅ Tabela 'chaves_pix' criada com sucesso!
📝 Criando tabela: transacoes
✅ Tabela 'transacoes' criada com sucesso!

🎉 CRIAÇÃO DE TABELAS CONCLUÍDA!
✅ Tabela 'transacoes' criada com sucesso!

🎉 CRIAÇÃO DE TABELAS CONCLUÍDA!


In [27]:
# Verify Table Creation
def verify_tables():
    """Verifica se todas as tabelas foram criadas corretamente"""
    
    try:
        conn = get_target_connection()
        
        # Lista todas as tabelas no schema
        list_tables_sql = """
            SELECT table_name, table_type
            FROM information_schema.tables 
            WHERE table_schema = %s
            ORDER BY table_name
        """
        
        cursor = conn.cursor()
        cursor.execute(list_tables_sql, (TARGETSCHEMA,))
        tables = cursor.fetchall()
        
        print(f"\n🔍 VERIFICAÇÃO DE TABELAS NO SCHEMA '{TARGETSCHEMA}'")
        print("=" * 60)
        
        if tables:
            print(f"📊 Total de tabelas encontradas: {len(tables)}")
            print("\nTabelas criadas:")
            
            for table_name, table_type in tables:
                print(f"✅ {table_name} ({table_type})")
                
                # Verifica colunas de cada tabela
                columns_sql = """
                    SELECT column_name, data_type, is_nullable, column_default
                    FROM information_schema.columns 
                    WHERE table_schema = %s AND table_name = %s
                    ORDER BY ordinal_position
                """
                
                cursor.execute(columns_sql, (TARGETSCHEMA, table_name))
                columns = cursor.fetchall()
                
                print(f"   📋 Colunas ({len(columns)}):")
                for col_name, data_type, nullable, default in columns:
                    null_info = "NULL" if nullable == "YES" else "NOT NULL"
                    default_info = f", DEFAULT: {default}" if default else ""
                    print(f"      • {col_name}: {data_type} ({null_info}{default_info})")
                print()
        else:
            print("⚠️ Nenhuma tabela encontrada!")
        
        cursor.close()
        conn.close()
        
        return len(tables)
        
    except Exception as e:
        logger.error(f"❌ Erro na verificação das tabelas: {e}")
        raise

# Executar verificação
table_count = verify_tables()


🔍 VERIFICAÇÃO DE TABELAS NO SCHEMA 'core_bank'
📊 Total de tabelas encontradas: 6

Tabelas criadas:
✅ agencias (BASE TABLE)
   📋 Colunas (6):
      • codigo_agencia: bigint (NOT NULL)
      • nome: character varying (NOT NULL)
      • endereco: text (NULL)
      • cidade: character varying (NOT NULL)
      • estado: character (NULL)
      • telefone: character varying (NULL)

✅ cartoes (BASE TABLE)
   📋 Colunas (5):
      • numero_cartao: character varying (NOT NULL)
      • id_cliente: bigint (NOT NULL)
      • data_emissao: timestamp without time zone (NULL, DEFAULT: CURRENT_TIMESTAMP)
      • limite_credito: numeric (NULL, DEFAULT: 0)
      • status: character (NULL, DEFAULT: 'A'::bpchar)

✅ chaves_pix (BASE TABLE)
   📋 Colunas (6):
      • codigo_agencia: bigint (NOT NULL)
      • nome: character varying (NOT NULL)
      • endereco: text (NULL)
      • cidade: character varying (NOT NULL)
      • estado: character (NULL)
      • telefone: character varying (NULL)

✅ cartoes (BASE T

## ✅ Setup Concluído com Sucesso!

### 📊 Resumo da Execução:
- **Database:** `{TARGET_DB}` criado/verificado
- **Schema:** `{TARGETSCHEMA}` criado/verificado  
- **Tabelas:** {table_count} tabelas criadas com sucesso

### 🗂️ Estrutura Criada:
1. **agencias** - Dados das agências bancárias
2. **clientes** - Informações dos clientes
3. **contas** - Contas bancárias dos clientes
4. **cartoes** - Cartões de débito/crédito
5. **chaves_pix** - Chaves PIX cadastradas
6. **transacoes** - Transações financeiras

### 🚀 Próximos Passos:
1. **Execute o notebook:** `02_insert_main_data.ipynb` para inserir os dados principais
2. **Execute o notebook:** `03_insert_movement_data.ipynb` para inserir os dados de movimentações

### 📋 Observações:
- Todas as tabelas foram criadas com relacionamentos (Foreign Keys)
- Os tipos de dados foram definidos com base na análise dos CSVs
- Primary Keys e constraints foram aplicadas adequadamente
- O sistema está pronto para receber os dados!